In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils

In [3]:
import numpy as np
from utils import run_multiple_experiments, AUC, load_behance_features, maxsim, avgsim, avgsimtopk

In [4]:
tmp = load_behance_features('/mnt/workspace/Behance/Behance_Image_Features.b')

bytes: 2930692096
n_items: 178787


In [5]:
item_ids = tmp['ids']
item_id2index = tmp['id2index']
vgg19_original = tmp['featmat']

In [6]:
vgg19_pca100 = np.load('/mnt/workspace/Behance/PCA/VGG19-PCA100.npy')
vgg19_pca200 = np.load('/mnt/workspace/Behance/PCA/VGG19-PCA200.npy')
vgg19_pca300 = np.load('/mnt/workspace/Behance/PCA/VGG19-PCA300.npy')
vgg19_pca400 = np.load('/mnt/workspace/Behance/PCA/VGG19-PCA400.npy')
vgg19_pca500 = np.load('/mnt/workspace/Behance/PCA/VGG19-PCA500.npy')

In [7]:
vgg19_original /= np.reshape(np.linalg.norm(vgg19_original, axis=1), (-1, 1))
vgg19_pca100 /= np.reshape(np.linalg.norm(vgg19_pca100, axis=1), (-1, 1))
vgg19_pca200 /= np.reshape(np.linalg.norm(vgg19_pca200, axis=1), (-1, 1))
vgg19_pca300 /= np.reshape(np.linalg.norm(vgg19_pca300, axis=1), (-1, 1))
vgg19_pca400 /= np.reshape(np.linalg.norm(vgg19_pca400, axis=1), (-1, 1))
vgg19_pca500 /= np.reshape(np.linalg.norm(vgg19_pca500, axis=1), (-1, 1))

In [23]:
featmat_list = [
    vgg19_pca100,
    vgg19_pca200,
    vgg19_pca300,
    vgg19_pca400,
    vgg19_pca500,
    vgg19_original,
]
featmat_name_list = [
    'VGG19-PCA100',
    'VGG19-PCA200',
    'VGG19-PCA300',
    'VGG19-PCA400',
    'VGG19-PCA500',
    'VGG19-4096',
]
ks = [1, 2, 3, None]

In [32]:
def get_cosine_simfunc(featmat):
    return lambda i,j : np.dot(featmat[i], featmat[j])

def append_avgsimtopk(scorefunc_list, pairwise_simfunc, k):
    scorefunc_list.append(lambda i, js: avgsimtopk(i, js, pairwise_simfunc, k))

def append_scorefunc_and_name(scorefunc_list, scorefunc_name_list, pairwise_simfunc, pairwise_simfunc_tag, ks):
    for k in ks:
        if k == 1:
            scorefunc_list.append(lambda i, js: maxsim(i, js, pairwise_simfunc))
            scorefunc_name_list.append('%s-maxsim' % pairwise_simfunc_tag)
        elif k == None:
            scorefunc_list.append(lambda i, js: avgsim(i, js, pairwise_simfunc))
            scorefunc_name_list.append('%s-avgsim' % pairwise_simfunc_tag)
        else:
            assert k >= 2
            append_avgsimtopk(scorefunc_list, pairwise_simfunc, k)
            scorefunc_name_list.append('%s-avgsimtop%d' % (pairwise_simfunc_tag, k))

In [33]:
scorefunc_list = []
scorefunc_name_list = []
for featmat, featmat_name in zip(featmat_list, featmat_name_list):
    cosine_simfunc = get_cosine_simfunc(featmat)
    append_scorefunc_and_name(
        scorefunc_list, scorefunc_name_list,
        cosine_simfunc, '%s-cosine' % featmat_name, ks)

In [34]:
scorefunc_list, scorefunc_name_list

([<function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_avgsimtopk.<locals>.<lambda>>,
  <function __main__.append_scorefunc_and_name.<locals>.<la

In [35]:
def compute_AUC(train_items, test_pos_items, test_neg_items, scorefunc):
    
    train_idxs = [item_id2index[i] for i in train_items]
    test_pos_idxs = [item_id2index[i] for i in test_pos_items]
    test_neg_idxs = [item_id2index[i] for i in test_neg_items]
    
    # rank test items
    pos_idxs_set = set(test_pos_idxs)
    test_idxs = test_pos_idxs + test_neg_idxs
    test_idxs.sort(key=lambda i : scorefunc(i, train_idxs), reverse=True)
    
    # compute AUC
    relevant_positions = [pos for pos, i in enumerate(test_idxs) if i in pos_idxs_set]
    return AUC(relevant_positions, len(test_idxs))

In [36]:
def get_compute_AUC_func(scorefunc):
    def wrapper(train_items, test_pos_items, test_neg_items):
        return compute_AUC(train_items, test_pos_items, test_neg_items, scorefunc)
    return wrapper

In [37]:
compute_AUC_func_list = list(map(get_compute_AUC_func, scorefunc_list))

In [38]:
run_multiple_experiments(compute_AUC_func_list, scorefunc_name_list, '/mnt/workspace/Behance/ExperimentResults/')

----- starting experiment VGG19-PCA100-cosine-maxsim ----
experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/VGG19-PCA100-cosine-maxsim_aucs.npy
	 elapsed_seconds = 69.14, mean_AUC = 0.64044
----- starting experiment VGG19-PCA100-cosine-avgsimtop2 ----
experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/VGG19-PCA100-cosine-avgsimtop2_aucs.npy
	 elapsed_seconds = 87.67, mean_AUC = 0.64911
----- starting experiment VGG19-PCA100-cosine-avgsimtop3 ----
experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/VGG19-PCA100-cosine-avgsimtop3_aucs.npy
	 elapsed_seconds = 87.89, mean_AUC = 0.65280
----- starting experiment VGG19-PCA100-cosine-avgsim ----
experiment successfully finished: results saved to /mnt/workspace/Behance/ExperimentResults/VGG19-PCA100-cosine-avgsim_aucs.npy
	 elapsed_seconds = 72.53, mean_AUC = 0.66036
----- starting experiment VGG19-PCA200-cosine-maxsim ---